In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import pickle

In [2]:
def load_data(images_path, labels_path):
    images = []
    labels = []
    for filename in os.listdir(images_path):
        img_path = os.path.join(images_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))  
        
        label_file = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(labels_path, label_file)
        
        with open(label_path, 'r') as f:
            label_data = f.readlines()
        label = []
        for line in label_data:
            parts = line.split()
            x_min, y_min, x_max, y_max = map(float, parts[2:])
            if len(label) < 5:
                label.append([x_min, y_min, x_max, y_max])
        
        images.append(img)
        while len(label) < 5:
            label.append([0,0,0,0])
        labels.append(label)
    X = np.array(images)
    Y = np.array(labels, dtype=np.float32) 
    Y = Y.reshape((len(Y), 5 * 4))
    return X,Y 

In [3]:
images_path = 'data/images2'
labels_path = 'data/labels2'
X, y = load_data(images_path, labels_path)


In [ ]:
print(f"Toplam Resim Sayısı: {len(X)}")
print(f"Toplam Label Sayısı: {len(y)}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Toplam Resim Sayısı: 16733
Toplam Label Sayısı: 16733


In [ ]:
max_faces = 5
  
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(max_faces * 4, activation='sigmoid')  
])

c:\Users\emira\OneDrive\Masaüstü\WORK\Mentor\MentorApp\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_1/Cast:0", shape=(None,), dtype=float32). Expected shape (None, 224, 224, 3), but input has incompatible shape (None,)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None,), dtype=int32)
  • training=True
  • mask=None

In [ ]:
with open('faceCount.pkl', 'wb') as f:
    pickle.dump(model, f)